# Using a bi-lstm to sort a sequence of integers

In [1]:
import random
import string

import mxnet as mx
from mxnet import gluon, nd
import numpy as np

## Data Preparation

In [2]:
max_num = 999
dataset_size = 60000
seq_len = 5
split = 0.8
batch_size = 512
ctx = mx.gpu() if mx.context.num_gpus() > 0 else mx.cpu()

We are getting a dataset of **dataset_size** sequences of integers of length **seq_len** between **0** and **max_num**. We use **split*100%** of them for training and the rest for testing.


For example:

50 10 200 999 30

Should return

10 30 50 200 999

In [3]:
X = mx.random.uniform(low=0, high=max_num, shape=(dataset_size, seq_len)).astype('int32').asnumpy()
Y = X.copy()
Y.sort() #Let's sort X to get the target

In [4]:
print("Input {}\nTarget {}".format(X[0].tolist(), Y[0].tolist()))

Input [548, 592, 714, 843, 602]
Target [548, 592, 602, 714, 843]


For the purpose of training, we encode the input as characters rather than numbers

In [5]:
vocab = string.digits + " "
print(vocab)
vocab_idx = { c:i for i,c in enumerate(vocab)}
print(vocab_idx)

0123456789 
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, ' ': 10}


We write a transform that will convert our numbers into text of maximum length **max_len**, and one-hot encode the characters.
For example:

"30 10" corresponding indices are [3, 0, 10, 1, 0]

We then one hot encode that and get a matrix representation of our input. We don't need to encode our target as the loss we are going to use support sparse labels

In [6]:
max_len = len(str(max_num))*seq_len+(seq_len-1)
print("Maximum length of the string: %s" % max_len)

Maximum length of the string: 19


In [7]:
def transform(x, y):
    x_string = ' '.join(map(str, x.tolist()))
    x_string_padded = x_string + ' '*(max_len-len(x_string))
    x = [vocab_idx[c] for c in x_string_padded]
    y_string = ' '.join(map(str, y.tolist()))
    y_string_padded = y_string + ' '*(max_len-len(y_string))
    y = [vocab_idx[c] for c in y_string_padded]
    return mx.nd.one_hot(mx.nd.array(x), len(vocab)), mx.nd.array(y)

In [8]:
split_idx = int(split*len(X))
train_dataset = gluon.data.ArrayDataset(X[:split_idx], Y[:split_idx]).transform(transform)
test_dataset = gluon.data.ArrayDataset(X[split_idx:], Y[split_idx:]).transform(transform)

In [9]:
print("Input {}".format(X[0]))
print("Transformed data Input {}".format(train_dataset[0][0]))
print("Target {}".format(Y[0]))
print("Transformed data Target {}".format(train_dataset[0][1]))

Input [548 592 714 843 602]
Transformed data Input 
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 19x11 @cpu(0)>
Target [548 592 602 714 843]
Transformed data Target 
[ 5.  4.  8. 10.  5.  9.  2. 10.  6.  0.  2. 10.  7.  1.  4. 10.  8.  4.
  3.]
<NDArray 19 @cpu(0)>


In [10]:
train_data = gluon.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=20, last_batch='rollover')
test_data = gluon.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=5, last_batch='rollover')

## Creating the network

In [11]:
net = gluon.nn.HybridSequential()
with net.name_scope():
    net.add(
        gluon.rnn.LSTM(hidden_size=128, num_layers=2, layout='NTC', bidirectional=True),
        gluon.nn.Dense(len(vocab), flatten=False)
    )

In [12]:
net.initialize(mx.init.Xavier(), ctx=ctx)

In [13]:
loss = gluon.loss.SoftmaxCELoss()

We use a learning rate schedule to improve the convergence of the model

In [14]:
schedule = mx.lr_scheduler.FactorScheduler(step=len(train_data)*10, factor=0.75)
schedule.base_lr = 0.01

In [15]:
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate':0.01, 'lr_scheduler':schedule})

## Training loop

In [16]:
epochs = 100
for e in range(epochs):
    epoch_loss = 0.
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)

        with mx.autograd.record():
            output = net(data)
            l = loss(output, label)

        l.backward()
        trainer.step(data.shape[0])
    
        epoch_loss += l.mean()
        
    print("Epoch [{}] Loss: {}, LR {}".format(e, epoch_loss.asscalar()/(i+1), trainer.learning_rate))

Epoch [0] Loss: 1.6627886372227823, LR 0.01
Epoch [1] Loss: 1.210370733382854, LR 0.01
Epoch [2] Loss: 0.9692377131035987, LR 0.01
Epoch [3] Loss: 0.7976046623067653, LR 0.01
Epoch [4] Loss: 0.5714595343476983, LR 0.01
Epoch [5] Loss: 0.4458411196444897, LR 0.01
Epoch [6] Loss: 0.36039798817736035, LR 0.01
Epoch [7] Loss: 0.32665719377233626, LR 0.01
Epoch [8] Loss: 0.262064205702915, LR 0.01
Epoch [9] Loss: 0.22285924059279422, LR 0.0075
Epoch [10] Loss: 0.19018426854559717, LR 0.0075
Epoch [11] Loss: 0.1718730723604243, LR 0.0075
Epoch [12] Loss: 0.15736752171670237, LR 0.0075
Epoch [13] Loss: 0.14579375246737866, LR 0.0075
Epoch [14] Loss: 0.13546599733068587, LR 0.0075
Epoch [15] Loss: 0.12490207590955368, LR 0.0075
Epoch [16] Loss: 0.11803316300915133, LR 0.0075
Epoch [17] Loss: 0.10653189395336395, LR 0.0075
Epoch [18] Loss: 0.10514750379197141, LR 0.0075
Epoch [19] Loss: 0.09590611559279422, LR 0.005625
Epoch [20] Loss: 0.08146028108494256, LR 0.005625
Epoch [21] Loss: 0.0770734

## Testing

We get a random element from the testing set

In [17]:
n = random.randint(0, len(test_data)-1)

x_orig = X[split_idx+n]
y_orig = Y[split_idx+n]

In [41]:
def get_pred(x):
    x, _ = transform(x, x)
    output = net(x.as_in_context(ctx).expand_dims(axis=0))

    # Convert output back to string
    pred = ''.join([vocab[int(o)] for o in output[0].argmax(axis=1).asnumpy().tolist()])
    return pred

Printing the result

In [43]:
x_ = ' '.join(map(str,x_orig))
label = ' '.join(map(str,y_orig))
print("X         {}\nPredicted {}\nLabel     {}".format(x_, get_pred(x_orig), label))

X         611 671 275 871 944
Predicted 275 611 671 871 944
Label     275 611 671 871 944


We can also pick our own example, and the network manages to sort it without problem:

In [66]:
print(get_pred(np.array([500, 30, 999, 10, 130])))

10 30 130 500 999  


The model has even learned to generalize to examples not on the training set

In [64]:
print("Only four numbers:", get_pred(np.array([105, 302, 501, 202])))

Only four numbers: 105 202 302 501    


However we can see it has trouble with other edge cases:

In [63]:
print("Small digits:", get_pred(np.array([10, 3, 5, 2, 8])))
print("Small digits, 6 numbers:", get_pred(np.array([10, 33, 52, 21, 82, 10])))

Small digits: 8  0 42 28         
Small digits, 6 numbers: 10 0 20 82 71 115  


This could be improved by adjusting the training dataset accordingly